# hotpot_qa

In [6]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/Users/ariete/Projects/self-improve/output/hotpot_qa/200_critic.jsonl", split="train")
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['idx', 'question', 'predictions', 'answer'],
    num_rows: 200
})


In [ ]:
import numpy as np
from utils.qa import em_f1_score, normalize_answer
em_scores = []
f1_scores = []
precision_scores = []
recall_scores = []
for idx, sample in enumerate(dataset):
        

    cur_em = []
    cur_f1 = []
    cur_precesion = []
    cur_recall = []

    predictions = [prediction.split("FINAL ANSWER:")[-1].strip() for prediction in sample["predictions"]]

    while len(predictions) < 5:
        predictions.append(predictions[-1])
    print(normalize_answer(sample['answer']))
    print([normalize_answer(prediction) for prediction in predictions])
    print()

    for prediction in predictions:

        em, f1, precision, recall = em_f1_score(normalize_answer(prediction), normalize_answer(sample['answer']))

        cur_em.append(em)
        cur_f1.append(f1)
        cur_precesion.append(precision)
        cur_recall.append(recall)
    

    em_scores.append(cur_em)
    f1_scores.append(cur_f1)
    precision_scores.append(cur_precesion)
    recall_scores.append(cur_recall)
    

# output mean of each column of scores
em_means = np.array(em_scores).mean(axis=0)
em_means = np.round(em_means* 100, decimals=1)

f1_means = np.array(f1_scores).mean(axis=0)
f1_means = np.round(f1_means* 100, decimals=1)

precision_means = np.array(precision_scores).mean(axis=0)
precision_means = np.round(precision_means* 100, decimals=1)

recall_means = np.array(recall_scores).mean(axis=0)
recall_means = np.round(recall_means* 100, decimals=1)

print(f"em_means: {em_means}\n", f"f1_means: {f1_means}\n", f"precision_means: {precision_means}\n", f"recall_means: {recall_means}\n")


# gsm8k

In [8]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/Users/ariete/Projects/self-improve/output/gsm8k/200_critic.jsonl", split="train")
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['idx', 'question', 'predictions', 'answer', 'prediction'],
    num_rows: 200
})


In [9]:
import re
from typing import Union

def normalize_answer(answer: str):
    answer = str(answer)
    # number
    answer = answer.replace(",", "")
    digits = re.findall(r"-?\d+\.?\d*", answer)
    answer = digits[-1] if len(digits) > 0 else None
    return floatify_ans(answer)


def get_precision(gt_ans: float) -> int:
    precision = 5
    if '.' in str(gt_ans):
        precision = len(str(gt_ans).split('.')[-1])
    return precision


def finqa_equal(prediction: Union[bool, float, str],
                reference: Union[float, str],
                include_percentage: bool = True,
                is_close: float = False) -> bool:
    if prediction is None:
        return False
    elif type(prediction) == bool:
        # bool questions
        if prediction:
            return reference == 'yes'
        else:
            return reference == 'no'
    elif type(reference) == str or type(prediction) == str:
        # string questions
        return prediction == reference
    else:
        # number questions
        if include_percentage:
            gt_result = [reference / 100, reference, reference * 100]
        else:
            gt_result = [reference]
        for item in gt_result:
            try:
                if is_close:
                    if isclose(item, prediction, rel_tol=0.001):
                        return True
                precision = min(get_precision(prediction), get_precision(item))
                if round(prediction, precision) == round(item, precision):
                    return True
            except Exception:
                continue
        return False


def round_with_error(x):
    return round(x * 1e5) / 1e5


def floatify_ans(ans):
    """gsm8k"""
    if ans is None:
        return None
    elif type(ans) == dict:
        ans = list(ans.values())[0]
    elif type(ans) == bool:
        ans = ans
    elif type(ans) in [list, tuple]:
        if not ans:
            return None
        else:
            try:
                ans = float(ans[0])
            except Exception:
                ans = str(ans[0])
    else:
        try:
            ans = float(ans)
            ans = round_with_error(ans)
        except Exception:
            ans = str(ans)
    return ans

In [15]:
import numpy as np

scores = []

for idx, sample in enumerate(dataset):

    while len(sample["predictions"]) < 4:
        sample["predictions"].append(sample["predictions"][-1])
    
    is_correct = []
    for prediction in sample["predictions"]:
        is_correct.append(finqa_equal(normalize_answer(prediction), normalize_answer(sample["answer"])))
    # print(is_correct)
    scores.append(is_correct)

print(scores)

TypeError: 'int' object is not iterable